# Overview of Significance of Topic
- Batting speed in baseball
    - How does it contribute to the success of a team
    - Factors that contribute to/inhibit batting speed
    - How does the ability to predict/measure batting speed impact team/player performance predictions

# Formal Introduction to Topic


For our final project we chose to analyze data from the Open Biomechanics Project (OBP) driveline baseball research. The data provides a biomechanical breakdown of a baseball player's swing by measuring forces produced by different body parts in all three dimensions. The data was captured using a combination of ground force plates, a marker motion capture system, bat speed sensors, and ball flight monitors. As a result, the data available is rather robust, with time series force plate data in all three dimensions, hip, torso, shoulder, and hand velocities, as well as bat speed and ball flight data. In essence, every piece of information that is responsible for a baseball swing is accounted for. The idea behind the data is to pick apart a player's swing to discover which aspects they should focus on in training to produce a faster bat speed when swinging. 


What we hope to do is create a machine learning model that identifies the most important features of a player's swing when generating bat speed, and then use those features to accurately predict a player's bat speed. Our model can be beneficial to baseball players and coaches, as it can be used to address aspects of a player's swing that need to improve in order to generate more bat speed. If an athlete's true bat speed closely aligns with their predicted bat speed, this would mean that they are currently moving efficiently in the swing. In other words, they are transferring all of the energy they produce through the bat and into contact. On the other hand, if a hitter did not meet their predicted bat speed value, this would be indicative of mechanical inefficiency. As a result, a coach with access to this data and model would be able to bucket athletes into two main groups: strength-focused and skill-focused. Players with efficient movements would benefit from strength training to increase their baseline capabilities for bat speed, whereas players with high force production would benefit from a mechanical focus.

# Variable Overview
- Break down dataset
- Include images to explain different variables
    - ex. hip rotation

# [ BUILDING THE MODEL/ EXPLAINING THE PROCESS]

# Conclusion of Results

# Bias
- Who benefits, who doesn't
- gender discrimination, accesibility
- pull from Project Proposal

# Application
- How could this be used by other teams/sports/groups 




# TO DO:
- David to write a quick paragraph on “Overview of Significance of Topic”
    - Julia will edit and bulk up with outside data
- David and Cece work on completing “Variable Overview” on Thursday
    - Julia to pull in images that explain different variables 
        - Ex. hip rotation visual 

